<a href="https://colab.research.google.com/github/ToiznerD/CloudComputing/blob/main/user_interface_dor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [147]:
from zmq.constants import THREAD_AFFINITY_CPU_REMOVE
import json
from firebase import firebase
import IPython
from IPython.display import HTML
from google.colab import output, drive
from IPython.display import clear_output
from IPython.display import Markdown
import time
import ipywidgets as widgets
from google.colab.output import _publish as publish

def logo():
  display(IPython.display.HTML(''' <h1> Ipypy</h1> '''))

def checkAnswer(ans, buttons):
  cButton = None
  global qnum, container

  # Check if container is visible
  if container.layout.display == 'none':
    return

  # Find the correct answer
  for b in buttons:
    if b.description == questions[qnum]['correct']:
      cButton = b
  
  # Validate the clicked answer
  if ans.description == cButton.description:
    ans.add_class('correctAnswer')
    global rAnswers
    rAnswers += 1
  else:
    ans.add_class('wrongAnswer')
    cButton.add_class('correctAnswer')
    global wAnswers
    wAnswers += 1
  
  time.sleep(3)
  
  qnum += 1
  if qnum == len(questions): # in case of last question
    container.layout.display = 'none'
    rAnswersLabel = widgets.HTML(value=f'Correct Answers: {rAnswers}')
    rAnswersLabel.add_class('h5')
    wAnswersLabel = widgets.HTML(value=f'Wrong Answers: {wAnswers}')
    wAnswersLabel.add_class('h4')
    answersContainer = widgets.VBox([rAnswersLabel, wAnswersLabel], layout=widgets.Layout(align_items='center', height="250px"))
    display(answersContainer)
    return
  
  # Present next question
  questionLabel.value = questions[qnum]['question']
  qnumLabel.value = f'Question #{qnum+1}'
  for i,b in enumerate(buttons):
    ansStr = ansStr = f'ans{i+1}'
    b.description = questions[qnum][ansStr]
    b.remove_class("wrongAnswer")
    b.remove_class("correctAnswer")
    b.add_class("button")


# Get style
drive.mount('/content/drive', force_remount=True)
with open('/content/drive/My Drive/MyStyles/style.css') as fid:
  style = fid.read()

FBconn = firebase.FirebaseApplication('https://cloudcomputing-e1f5d-default-rtdb.firebaseio.com/',None)
res=FBconn.get('/Questions/',None)
questions = []
rAnswers=0
wAnswers=0
qnum = 0

for key in res:
    questions.append(res[key])

# Loading window

publish.css(style)
display(IPython.display.HTML('''
<html>
  <head>
    <h1> Ipypy</h1>
    <h2>level 1</h2>
  </head>
  <body>

    <!-- Loading bar -->
    <div id="loading-bar">
      <div id="loading-progress"></div>
    </div>

    <!-- JavaScript code to simulate loading the game -->
    <script>
      var progress = 0;
      var loading = setInterval(function() {
        progress += Math.floor(Math.random() * 10); // simulate progress
        document.getElementById('loading-progress').style.width = progress + '%';
        if (progress >= 100) {
          clearInterval(loading); // stop the interval when progress reaches 100%
        }
      }, 500);
    </script>
  </body>
</html>
'''))

time.sleep(2)

# Clear loading window
clear_output(wait=True)

# Start game
publish.css(style)

# Present logo
logo()

button_layout = {'width': '350px','height': '100px', 'margin': '10px'}
buttons = []
for i in range(1,5):
  ansStr = f'ans{i}'
  b = widgets.Button(description=questions[qnum][ansStr], layout=button_layout)
  b.add_class("button")
  buttons.append(b)

button_container = widgets.HBox(children=buttons)

# Add listeners
for b in buttons:
  b.on_click(lambda button: checkAnswer(button, buttons))


# Wrap the button container in a VBox container and center it horizontally
qnumLabel = widgets.HTML(value=f'Question #{qnum+1}')
qnumLabel.add_class("h2")
questionLabel = widgets.HTML(value=questions[0]['question'])
questionLabel.add_class("h2")

container = widgets.VBox([qnumLabel, questionLabel, button_container], layout=widgets.Layout(align_items='center'))
display(container)
